In [1]:
import pandas as pd
import pickle
import psycopg2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
matplotlib.use('TkAgg')

In [2]:
# Load the model
with open('random_forest_model.pkl', 'rb') as file:
    model = pickle.load(file)
print('DONE')

DONE


In [3]:

user_profile = {
    'professional_review':1, 
    'vibe_review': 0, 
    'redness_review': 0, 
    'dry_review': 0, 
    'light_coverage_review': 0, 
    'young_review': 1, 
    'mother_review': 0, 
    'skin_concerns_review': 0, 
    'white_review': 1, 
    'tan_review': 1, 
    'acne_review': 0,
    'poc_review': 0, 
    'comfortable_wear_review': 0, 
    'coverage_review': 0, 
    'medium_coverage_review': 0, 
    'full_coverage_review': 1, 
    'easy_use_review': 0, 
    'wrinkles_review': 0
}
product_data = pd.read_csv('product_embeddings.csv')
product_copy = product_data.copy()
product_copy = product_copy.drop(columns = ['product_link_id', 'overall_product_rating'])
le = LabelEncoder()
product_copy['category'] = le.fit_transform(product_copy['category'])

print(product_copy.columns)






Index(['young', 'mother', 'professional', 'vibe', 'acne', 'dry', 'wrinkles',
       'poc', 'white', 'tan', 'redness', 'coverage', 'light_coverage',
       'medium_coverage', 'full_coverage', 'expensive', 'inexpensive',
       'skin_concerns', 'comfortable_wear', 'easy_use', 'pilling',
       'shade_range', 'category', 'num_shades', 'num_reviews'],
      dtype='object')


In [4]:
for col, val in user_profile.items(): 
    product_copy[col] = user_profile[col]



In [5]:
print(product_copy.head(1))

      young    mother  professional      vibe      acne       dry  wrinkles  \
0  0.684078  0.821281      0.835244  0.693004  4.840942  0.953626  0.743352   

        poc     white       tan  ...  white_review  tan_review  acne_review  \
0  0.938023  0.951343  0.924662  ...             1           1            0   

   poc_review  comfortable_wear_review  coverage_review  \
0           0                        0                0   

   medium_coverage_review  full_coverage_review  easy_use_review  \
0                       0                     1                0   

   wrinkles_review  
0                0  

[1 rows x 43 columns]


In [6]:

feature_names = model.feature_names_in_  # This assumes scikit-learn's RandomForest model
print(model.classes_)
# Reorder the DataFrame to match the model's expected feature order
product_copy = product_copy[feature_names]
predictions = model.predict_proba(product_copy)
product_copy['predicted_score'] = predictions[:, 4]
print(predictions[0, :])
product_copy['product_link_id'] = product_data['product_link_id']
top_products = product_copy.sort_values(by='predicted_score', ascending=False)
# Output the top products
top_n = 10  # Number of top products to display
print(top_products[['product_link_id', 'predicted_score']].head(top_n))

[1 2 3 4 5]
[0.14       0.11       0.17       0.23018315 0.34981685]
      product_link_id  predicted_score
387             406.0         0.570000
389             408.0         0.570000
184             197.0         0.484077
187             200.0         0.477143
294             311.0         0.473333
1102           1179.0         0.471739
2                 3.0         0.471739
181             193.0         0.471739
13               15.0         0.471538
1282           1366.0         0.470000


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished
